In [1]:
import pandas as pd
import geopandas as gpd
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.multioutput import MultiOutputRegressor
import warnings
warnings.filterwarnings('ignore')

### Helper Functions

In [2]:
def loadData(file):
    data = pd.read_csv(file)
    print('Raw shape: ',data.shape)
    data['Date'] = pd.to_datetime(data.Date)
    print('Days: ',len(set(data.Date)))
    return data

In [3]:
def getTimeSeries(df,zones):
    df = df.loc[df['DOLocationID'].isin(zones)]
    table = pd.pivot_table(df, values='vehicle_count', index=['Date','Hour'],
                    columns=['DOLocationID'], aggfunc=np.sum, fill_value=0)
#     table.columns = [i[1] for i in table.columns]
    missing_columns = [i for i in zones if i not in table.columns]
    for col in missing_columns:
        table[col] = 0
    table = table[sorted(table.columns)]
    return table

In [4]:
def zscoreNormalizeSpatial(matrix):
    m = matrix.copy()
    for i in range(m.shape[0]):
        m[i, :] = (m[i, :] - m[i, :].mean()) / (m[i, :].std()+1e-10)
        
    return m

In [5]:
def standardize(matrix):
    m = matrix.copy()
    scaler = StandardScaler()
    scaler.fit(m)
    t = scaler.transform(m)
    return scaler, t
def inverse_standardize(matrix, scaler):
    t = matrix.copy()
    return scaler.inverse_transform(t)
def getPCAFeatures(matrix, n=10):
    pca = PCA(n_components=n)
    pca.fit(matrix)
    reducedMatrixPCA = pca.transform(matrix)
    reducedMatrixPCA.shape

    reducedDict = {str(i+1):reducedMatrixPCA[:,i] for i in range(reducedMatrixPCA.shape[1])}
    reducedDf = pd.DataFrame(reducedDict)
    #reducedDf.index = index
    return pca,reducedDf

In [6]:
def inverse_standardize(matrix, scaler):
    t = matrix.copy()
    return scaler.inverse_transform(t)

In [7]:
def getPCAFeatures(matrix, n=10):
    pca = PCA(n_components=n)
    pca.fit(matrix)
    reducedMatrixPCA = pca.transform(matrix)
    reducedMatrixPCA.shape

    reducedDict = {str(i+1):reducedMatrixPCA[:,i] for i in range(reducedMatrixPCA.shape[1])}
    reducedDf = pd.DataFrame(reducedDict)
    #reducedDf.index = index
    return pca,reducedDf

In [8]:
def inverse_pca(matrix,pca):
    m = matrix.copy()
    return pca.inverse_transform(m)

In [9]:
def addLag(dataset, maxlag, lagColumns):
    dataset_list = [dataset]

    for l in range(1, maxlag+1):
        df = dataset.shift(l)
        df = df[lagColumns]
        df.columns = [c+'_lag_'+str(l) for c in df.columns]
        dataset_list.append(df)

    dataset = pd.concat(dataset_list, axis=1).dropna()
    return dataset

#### train PCA from 2018

In [10]:
zone = gpd.read_file('../../Data/NYC Taxi Zones.geojson')
zones = zone['location_id'].unique()
zones = [int(i) for i in zones]

In [11]:
len(zones)

260

In [12]:
hub = 'JFK'
pca_comps = 6
dataDir = '/home/mingyi/Dropbox/UrbanTemporalNetworks/processedData/'
file = dataDir + hub + 'VehicleByHour2018fromHub.csv'

In [13]:
data2018 = loadData(file)
data2018 = getTimeSeries(data2018,zones)
data2018.head()

Raw shape:  (2268840, 4)
Days:  365


DOLocationID     1    2    3    4    5    6    7    8    9    10   ...  254  \
Date       Hour                                                    ...        
2018-01-01 0       1    0    0    1    0    0    5    0    0    7  ...    0   
           1       0    0    1    1    0    0    1    0    1    4  ...    0   
           2       0    0    0    1    0    0    1    0    0    1  ...    0   
           3       0    0    0    0    0    0    0    0    0    0  ...    0   
           4       0    0    0    0    0    0    0    0    0    3  ...    0   

DOLocationID     255  256  257  258  259  260  261  262  263  
Date       Hour                                               
2018-01-01 0       2    4    0    0    2    3    0    5    6  
           1       2    2    1    1    0    2    0    0    1  
           2       0    1    0    1    0    1    0    2    0  
           3       1    0    0    2    0    0    0    0    1  
           4       0    1    0    1    0    0    0    0    0  

[5 rows x 260 columns]

In [14]:
matrix2018 = data2018.values.astype(np.float64)
scaler, s_matrix2018 = standardize(matrix2018)
pca,pca_data2018 = getPCAFeatures(s_matrix2018,n=pca_comps)

In [15]:
r2_score(s_matrix2018,pca.inverse_transform(pca_data2018))

0.3238709760099891

#### Preparing Data

In [16]:
hub = 'JFK'
file = dataDir + hub + 'VehicleByHour2019fromHub.csv'

In [17]:
data = loadData(file)
data = getTimeSeries(data,zones)
data.shape

Raw shape:  (2295120, 4)
Days:  365


(8760, 260)

In [18]:
matrix = data.values.astype(np.float64)
scaler, s_matrix = standardize(matrix)
pcaData = pd.DataFrame(pca.transform(s_matrix),columns=[str(i) for i in range(1,pca_comps+1)])

In [19]:
r2_score(s_matrix,pca.inverse_transform(pcaData))

0.3232405656159446

In [20]:
pcaData.index = data.index
pcaData = pcaData.reset_index()

In [21]:
externalDataDir = "/home/mingyi/Dropbox/UrbanTemporalNetworks/HongData/"+hub+'2019/'
extFile = externalDataDir+"external.csv"

In [22]:
extDf = pd.read_csv(extFile)
print(extDf.shape)
extDf.head(2)

(8760, 14)


,Date,Hour,arrival,PRCP,SNOW,SNWD,TMAX,DOW,Tue,Wed,Thur,Fri,Sat,Sun
0,2019-01-01,0,7.0,0.06,0.0,0.0,58.0,1,1,0,0,0,0,0
1,2019-01-01,1,1.0,0.06,0.0,0.0,58.0,1,1,0,0,0,0,0


In [23]:
extDf.columns

Index(['Date', 'Hour', 'arrival', 'PRCP', 'SNOW', 'SNWD', 'TMAX', 'DOW', 'Tue',
       'Wed', 'Thur', 'Fri', 'Sat', 'Sun'],
      dtype='object')

In [24]:
selected_columns = ['Date', 'Hour', 'arrival', 'PRCP', 'SNOW', 'SNWD', 'TMAX', 'DOW', 'Tue',
       'Wed', 'Thur', 'Fri', 'Sat', 'Sun']

In [25]:
extDf = extDf[selected_columns]

In [26]:
pca_data2018

,1,2,3,4,5,6
0,-3.588562,4.393972,1.528272,0.915964,-0.485206,-0.862172
1,-7.963274,3.220824,0.531684,1.292945,-1.604876,0.405257
2,-12.807477,0.654048,1.848285,-0.139043,-0.503030,0.231892
3,-13.669137,0.381856,1.661397,-0.075564,-0.369869,0.188619
4,-12.983212,0.557505,1.068529,-0.578112,0.405943,-0.155874
...,...,...,...,...,...,...
8755,11.880840,4.607175,2.461575,1.045820,0.689446,-3.027006
8756,8.048243,5.494914,3.000300,-0.447031,-2.832418,-2.472481
8757,9.078574,7.724551,3.807493,-0.502671,-1.242993,-0.844255
8758,7.548258,8.058080,0.622593,0.775275,0.450685,-0.787450


In [27]:
print(pcaData.shape)
print(extDf.shape)

(8760, 8)
(8760, 14)


In [28]:
pcaData['Date'] = pd.to_datetime(pcaData['Date'])
extDf['Date'] = pd.to_datetime(extDf['Date'])
pcaData['Hour'] = pcaData['Hour'].astype(int)
extDf['Hour'] = extDf['Hour'].astype(int)

In [29]:
pcaData = pd.merge(pcaData,extDf, on=['Date', 'Hour'], how='left')
print(pcaData.shape)

(8760, 20)


In [30]:
pcaData.columns

Index(['Date', 'Hour', '1', '2', '3', '4', '5', '6', 'arrival', 'PRCP', 'SNOW',
       'SNWD', 'TMAX', 'DOW', 'Tue', 'Wed', 'Thur', 'Fri', 'Sat', 'Sun'],
      dtype='object')

In [31]:
lagColumns = ['arrival'] + [str(i) for i in range(1,1+pca_comps)]

DateColumns = ['Date']

targetColumns = [str(i) for i in range(1,1+pca_comps)]

In [32]:
maxlag = 12

pcaData_lag = addLag(pcaData, maxlag, lagColumns)

pcaData_lag.shape

(8748, 104)

In [33]:
### 2018 data with externality
pca_data2018.index = data2018.index
pca_data2018 = pca_data2018.reset_index()
externalDataDir = "/home/mingyi/Dropbox/UrbanTemporalNetworks/HongData/"+hub+'2018/'
extFile = externalDataDir+"external.csv"
extDf = pd.read_csv(extFile)
extDf = extDf[selected_columns]

pca_data2018['Date'] = pd.to_datetime(pca_data2018['Date'])
extDf['Date'] = pd.to_datetime(extDf['Date'])
pca_data2018['Hour'] = pca_data2018['Hour'].astype(int)
extDf['Hour'] = extDf['Hour'].astype(int)

pca_data2018 = pd.merge(pca_data2018,extDf, on=['Date', 'Hour'], how='left')

pca_data2018_lag = addLag(pca_data2018, maxlag, lagColumns)


In [35]:
CommR2List = []
EdgeR2List = []
residualDf_list = []
rawList = []
networkPrediction = pd.DataFrame()

for m in range(1,13):
    print()

    print("month: ",m)
    month_index  = pd.to_datetime(pcaData_lag.Date).dt.month == m

    dataset_train = pcaData_lag[~month_index]
    dataset_test = pcaData_lag[month_index]
    print("Train Size: ",dataset_train.shape)
    print("Test Size: ",dataset_test.shape)


    X_train = dataset_train.drop(targetColumns+DateColumns , axis = 1)
    X_test = dataset_test.drop(targetColumns+DateColumns , axis = 1)
    y_train = dataset_train[targetColumns]
    y_test = dataset_test[targetColumns]



    val_size = int(X_train.shape[0]*0.8)
    val_fold = list(-1*np.ones(X_train.shape[0]-val_size)) + list(np.zeros(val_size))
    ps = PredefinedSplit(val_fold)
    param_grid = [{
        "n_estimators": np.arange(10, 500, 50),
        "min_samples_split": np.arange(2, 50, 20),
        'min_samples_leaf': np.arange(2, 50, 20), 
        'max_features': ['sqrt'],
        'max_depth': np.arange(10, 50, 10),
    }]
    # fit_inverse_transform=True to make sure inverse transform available
    rf = RandomForestRegressor(random_state = 2019) 
    rf_grid_search = GridSearchCV(rf, param_grid, cv=ps, scoring='r2')
    rf_grid_search.fit(X_train, y_train)

    print("Train R2: ",rf_grid_search.best_estimator_.score(X_train,y_train))
    test_r2 = rf_grid_search.best_estimator_.score(X_test,y_test)
    print("Test R2: ",test_r2)


    pca_prediction = rf_grid_search.best_estimator_.predict(X_test)

    residual = y_test - pca_prediction
    residual_df = dataset_test[['Date','Hour']]
    residual_df = pd.concat([residual_df,pd.DataFrame(residual)], axis =1)

    network_prediction = inverse_pca(pca_prediction,pca)

    network_prediction = inverse_standardize(network_prediction, scaler)
    
    # relu to convert all prediction to positive
#     network_prediction = np.log(1+np.e**network_prediction)
    # round up negative values to 0
#     network_prediction = np.where(network_prediction<0,0,network_prediction)

    network_prediction_df = pd.DataFrame(network_prediction)
    network_prediction_df.columns = data.columns
    networkPrediction = pd.concat([networkPrediction,network_prediction_df])
    edgeMonthIndex = [False] * maxlag + list(month_index)
    edge_r2 = r2_score(data[edgeMonthIndex], network_prediction, multioutput='variance_weighted')
    print("Edge R2: ",edge_r2)


    CommR2List.append(test_r2)
    EdgeR2List.append(edge_r2)
    residualDf_list.append(residual_df)
#     rawList.append()


month:  1
Train Size:  (8016, 104)
Test Size:  (732, 104)
Train R2:  0.9474907003418086
Test R2:  0.7602312393589121
Edge R2:  0.4970566836001448

month:  2
Train Size:  (8076, 104)
Test Size:  (672, 104)
Train R2:  0.9465226394642942
Test R2:  0.7806003984223646
Edge R2:  0.5365418100044741

month:  3
Train Size:  (8004, 104)
Test Size:  (744, 104)
Train R2:  0.9463637056454515
Test R2:  0.8386673089576178
Edge R2:  0.585503625407202

month:  4
Train Size:  (8028, 104)
Test Size:  (720, 104)
Train R2:  0.9489299379739523
Test R2:  0.7789686031253396
Edge R2:  0.556634956110667

month:  5
Train Size:  (8004, 104)
Test Size:  (744, 104)
Train R2:  0.9478686673958027
Test R2:  0.816198516628748
Edge R2:  0.4863003427315099

month:  6
Train Size:  (8028, 104)
Test Size:  (720, 104)
Train R2:  0.9481372813839379
Test R2:  0.8036782043017064
Edge R2:  0.5308021788312349

month:  7
Train Size:  (8004, 104)
Test Size:  (744, 104)
Train R2:  0.9485424325764092
Test R2:  0.7676145293256248
Edg

In [40]:
network_real_value = data[12:].reset_index()

In [43]:
CommR2List = []
EdgeR2List = []
residualDf_list = []
rawList = []
networkPrediction = pd.DataFrame()

for m in range(1,13):

    print("month: ",m)

    dataset_train = pd.concat([pca_data2018_lag.loc[pca_data2018_lag.Date.dt.month>=m],
                               pcaData_lag.loc[pcaData_lag.Date.dt.month<m]])
    dataset_test = pcaData_lag.loc[pcaData_lag.Date.dt.month==m]
    print("Train Size: ",dataset_train.shape)
    print("Test Size: ",dataset_test.shape)


    X_train = dataset_train.drop(targetColumns+DateColumns , axis = 1)
    X_test = dataset_test.drop(targetColumns+DateColumns , axis = 1)
    y_train = dataset_train[targetColumns]
    y_test = dataset_test[targetColumns]



    val_last_year = len(dataset_train.loc[dataset_train.Date.dt.month==m])
    val_last_month = len(dataset_train.loc[dataset_train.Date.dt.month==m-1])
    if m == 1:
        val_last_month = len(pca_data2018_lag.loc[pca_data2018_lag.Date.dt.month==12])
#     val_fold = list(np.zeros(val_last_year)) +\
#                 list(-1*np.ones(len(dataset_train)-val_last_year-val_last_month)) +\
#                 list(np.zeros(val_last_month))
    val_fold = list(-1*np.ones(len(dataset_train)-val_last_month)) +\
                list(np.zeros(val_last_month))
    ps = PredefinedSplit(val_fold)
    param_grid = [{
        "n_estimators": np.arange(10, 200, 50),
        "min_samples_split": np.arange(2, 50, 20),
        'min_samples_leaf': np.arange(2, 50, 20), 
        'max_features': ['sqrt'],
        'max_depth': np.arange(10, 50, 10),
    }]
    # fit_inverse_transform=True to make sure inverse transform available
    rf = RandomForestRegressor(random_state = 2019) 
    rf_grid_search = GridSearchCV(rf, param_grid, cv=ps, scoring='r2')
    rf_grid_search.fit(X_train, y_train)

    print("Train R2: ",rf_grid_search.best_estimator_.score(X_train,y_train))
    test_r2 = rf_grid_search.best_estimator_.score(X_test,y_test)
    print("Test R2: ",test_r2)


    pca_prediction = rf_grid_search.best_estimator_.predict(X_test)

    residual = y_test - pca_prediction
    residual_df = dataset_test[['Date','Hour']]
    residual_df = pd.concat([residual_df,pd.DataFrame(residual)], axis =1)

    network_prediction = inverse_pca(pca_prediction,pca)

    network_prediction = inverse_standardize(network_prediction, scaler)
    
    # relu to convert all prediction to positive
#     network_prediction = np.log(1+np.e**network_prediction)
    # round up negative values to 0
#     network_prediction = np.where(network_prediction<0,0,network_prediction)

    network_prediction_df = pd.DataFrame(network_prediction)
    network_prediction_df.columns = data.columns
    networkPrediction = pd.concat([networkPrediction,network_prediction_df])

    edge_r2 = r2_score(network_real_value.loc[network_real_value.Date.dt.month==m].drop(columns=['Date','Hour']).values, 
                       network_prediction, multioutput='variance_weighted')
    print("Edge R2: ",edge_r2)


    CommR2List.append(test_r2)
    EdgeR2List.append(edge_r2)
    residualDf_list.append(residual_df)
#     rawList.append()

month:  1
Train Size:  (8748, 104)
Test Size:  (732, 104)
Train R2:  0.9465393022271196
Test R2:  0.770449773773492
Edge R2:  0.5102841139426799
month:  2
Train Size:  (8748, 104)
Test Size:  (672, 104)
Train R2:  0.9466766902432138
Test R2:  0.8041052517723392
Edge R2:  0.554190072390432
month:  3
Train Size:  (8748, 104)
Test Size:  (744, 104)
Train R2:  0.9484760461165993
Test R2:  0.8394898626703631
Edge R2:  0.5854219126247516
month:  4
Train Size:  (8748, 104)
Test Size:  (720, 104)
Train R2:  0.9476480208232959
Test R2:  0.7849850931937852
Edge R2:  0.55561770701177
month:  5
Train Size:  (8748, 104)
Test Size:  (744, 104)
Train R2:  0.9482688083503775
Test R2:  0.8242543326702986
Edge R2:  0.48821000138264914
month:  6
Train Size:  (8748, 104)
Test Size:  (720, 104)
Train R2:  0.9482046455680097
Test R2:  0.8038700505931252
Edge R2:  0.5296313471381865
month:  7
Train Size:  (8748, 104)
Test Size:  (744, 104)
Train R2:  0.946252694313724
Test R2:  0.7677801213114418
Edge R2:  0

In [45]:
networkPrediction['Date'] = data.reset_index().iloc[12:]['Date'].values
networkPrediction['Hour'] = data.reset_index().iloc[12:]['Hour'].values
networkPrediction.to_csv('/home/mingyi/Dropbox/UrbanTemporalNetworks/prediction/learnFrom2017/from%sPCA%s2019.csv'%(hub,pca_comps),index=False)

In [44]:
print(np.mean(CommR2List))
print(np.mean(EdgeR2List))

0.8074695621777819
0.5376483959443491


In [46]:
networkPrediction[zones][networkPrediction[zones]<0] = 0
networkPrediction.to_csv('/home/mingyi/Dropbox/UrbanTemporalNetworks/prediction/learnFrom2017/from%sPCA%s2019Round.csv'%(hub,pca_comps),index=False)

In [58]:
res_df = pd.concat(residualDf_list, axis = 0)
res_df.to_csv('../../Resid/'+hub+'PCARoundup'+str(pca_comps)+'RFCV.csv',index=False)